In [2]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import pickle
from tqdm import tqdm
from time import sleep

from torchvision.models import vgg16, VGG16_Weights
from torchvision import transforms
from PIL import Image
import torch

from helpers import *
import database as Database

In [ ]:
# Main
card_dirs = os.listdir("cards/")

create_homography_for_all_cards(card_dirs)
calculate_features_for_all_cards(card_dirs)

# THIS WILL RESET THE DATABASe
database = Database.Database()
# THE ABOVE WILL RESET THE DATABASE

#database = load_database()
database.add_new_cards()
save_database(database)

In [3]:
# Reload Database Object and save
sorted_cards = database.sorted_cards
card_groups = database.card_groups
database = Database.Database()
database.sorted_cards = sorted_cards
database.card_groups = card_groups
save_database(database)

In [4]:
def get_group_to_cards(database):
    group_to_cards = {}
    for k,v in database.card_groups.items():
        if v in group_to_cards:
            group_to_cards[v].append(k)
        else:
            group_to_cards[v] = [k]

    return group_to_cards

def visualise_group(database, group, group_to_cards):
    group_i = group_to_cards[group]
    for i, path in enumerate(group_i):
        plt.subplot(1, len(group_i), i+1)
        image = Image.open(f"homography_cards/{path}")
        plt.axis("off")
        plt.imshow(image)
    
    plt.show()

In [ ]:
# Show all copies of multiple cards
group_to_cards = get_group_to_cards(database)
multi_card_groups = [k for k,v in group_to_cards.items() if len(v) > 1]
print(f"There are {len(multi_card_groups)} cards that you have duplicates of:")
for group in multi_card_groups:
    print(group_to_cards[group])
    visualise_group(database, group, group_to_cards)

In [3]:
# Create Features for all cards scraped from online
from helpers import extract_features, create_model_and_preprocess

model, preprocess = create_model_and_preprocess()

try:
    all_sets = os.listdir("card_db/sets")
    for set in tqdm(all_sets):

        # make a directory for the set if it doesn't exist
        if not os.path.exists(os.path.join(os.getcwd(), f"card_db\\features", set)):
            os.makedirs(f"card_db/features/{set}")
        
        # make a directory for the set if it doesn't exist
        if not os.path.exists(os.path.join(os.getcwd(), f"card_db\\homography_cards", set)):
            os.makedirs(f"card_db/homography_cards/{set}")

        # Loop through all cards in a set
        all_cards = os.listdir(f"card_db/sets/{set}")
        for card in all_cards:
            
            # If we have already computed the features then skip this card
            if os.path.exists(os.path.join(os.getcwd(), f"card_db\\features\\{set}", f"{card[:-4]}.pkl")):
                continue
            
            # Add a border and rotate the image a bit, otherwise we get infinite gradient lines
            img = cv.imread(f"card_db/sets/{set}/{card}")
            border_size = 20
            border_color = [0,0,0]
            img = cv.copyMakeBorder(img, border_size, border_size, border_size, border_size, cv.BORDER_CONSTANT, value=border_color)

            # rotation matrix
            height, width = img.shape[:2]
            center = (width // 2, height // 2)
            angle = 5
            M = cv.getRotationMatrix2D(center, angle, 1.0)

            # compute the new image size
            cos = np.abs(M[0, 0])
            sin = np.abs(M[0, 1])
            new_width = int((height * sin) + (width * cos))
            new_height = int((height * cos) + (width * sin))

            # adjust the rotation matrix to take into account the translation
            M[0, 2] += (new_width / 2) - center[0]
            M[1, 2] += (new_height / 2) - center[1]
            
            # perform the rotation
            img = cv.warpAffine(img, M, (new_width, new_height))

            # Find the corners of the card, compute the homography, and resave it
            _, edges = detect_edges(img)
            pts_src = corners_from_edges(edges)
            card_homography = compute_homography(pts_src, img)
            card_homography = cv.resize(card_homography, (300, 400), card_homography, interpolation=cv.INTER_AREA)
            cv.imwrite(f"card_db/homography_cards/{set}/{card}", card_homography)
            
            # Calculate the features of the card and dump them
            features = extract_features(model, preprocess, f"card_db/homography_cards/{set}/{card}")
            with open(f"card_db/features/{set}/{card[:-4]}.pkl", "wb") as file:
                pickle.dump(features, file)
except:
    print(set, card)

 52%|█████▏    | 78/151 [25:49<24:10, 19.87s/it]  

Legendary-Collection-Expansion Scoop-Up-Card-104.jpg


In [ ]:
# Delete the cards where there were no images on the website
model, preprocess = create_model_and_preprocess()

card_dir = "card_db/pokellector_empty.jpg"
img = cv.imread(card_dir)

all_sets = os.listdir("card_db/sets")
for set in tqdm(all_sets):

    # Loop through all cards in a set
    all_cards = os.listdir(f"card_db/sets/{set}")
    for card in all_cards:
        try:
            img2 = cv.imread(f"card_db/sets/{set}/{card}")
            if np.sum(img2-img) == 0:
                os.remove(f"card_db/sets/{set}/{card}")
                with open("card_db/blacklist.txt", "a") as file:
                    file.write(f"{set}/{card}")
        except:
            pass